# Predict Winner from Realtime Stats using RNN

In [12]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.autograd import Variable
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm


SEED = 233
BATCH_SIZE = 32
PREDICT_TIME = 10

## Dataset

In [13]:
class LOLDataset(Dataset):
    def __init__(self,moneyData,label):
        self.money=[]
        for i in range(PREDICT_TIME):
            self.money.append([line[i] for line in moneyData])
        self.label=[i for i in label]
    
    def __getitem__(self,item):
        return torch.tensor([[torch.scalar_tensor(x[item])] for x in self.money]),torch.tensor(self.label[item])
    
    def __len__(self):
        return len(self.label)
            

In [14]:
df = pd.read_csv('../data/LeagueofLegends.csv', sep=',')
df = df[df['gamelength'] >= PREDICT_TIME]
df.reset_index(drop = True, inplace = True)
print(f'# of matches: {len(df)}')

from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
df['golddiff'].head()
# df["bResult"]

# of matches: 7620


0    [0, 0, -14, -65, -268, -431, -488, -789, -494,...
1    [0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...
2    [0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...
3    [0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...
4    [40, 40, 44, -36, 113, 158, -121, -191, 23, 20...
Name: golddiff, dtype: object

Normalize data:

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for row in df['golddiff']:
    scaler.partial_fit(np.asanyarray(row).reshape(-1, 1))

golddiff = [scaler.transform(np.asanyarray(row).reshape(-1, 1)).reshape(-1) for row in df['golddiff']]
print(len(golddiff))

7620


Split the dataset into Train:Held-Out:Test = 6:2:2.

In [16]:
dataset = LOLDataset(golddiff, df["bResult"])
test_size = valid_size = int(0.2 * len(dataset))
train_size = len(dataset)- test_size - valid_size

trainDataset, validDataset, testDataset = random_split(
    dataset = dataset,
    lengths = [train_size, valid_size, test_size],
    generator = torch.Generator().manual_seed(SEED)
)

trainLoader = DataLoader(trainDataset, batch_size = BATCH_SIZE, shuffle=True)
validLoader = DataLoader(validDataset, batch_size = BATCH_SIZE)
testLoader = DataLoader(testDataset, batch_size = BATCH_SIZE)

## RNN Network

In [17]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()

        self.rnn=nn.LSTM(
            input_size = 1,
            hidden_size = 256,
            num_layers = 1,
            batch_first = True
        )

        self.out = nn.Linear(256, 2)
    
    def forward(self,x):
        r_out,(h_n, h_c)=self.rnn(x,None)
        out = self.out(r_out[:, -1, :])
        return out

In [18]:
model=RNN()
print(model)

RNN(
  (rnn): LSTM(1, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)


## Train

In [19]:
def train(dataloader, model, loss_fn, optimizer, mute = False):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = Variable(x), Variable(y)

        predict = model(x)
        loss = loss_fn(predict, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0 and not mute:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
def test(dataloader, model, loss_fn, validation = False):
    model.eval()
    size = len(dataloader.dataset)

    correct = 0
    test_loss = 0
    with torch.no_grad():
        for step,(x,y) in enumerate(dataloader):
            x, y = Variable(x), Variable(y)
            predict = model(x)
            # print(predict)
            test_loss += loss_fn(predict, y).item()
            correct += (predict.argmax(1) == y).sum().item()
    
    print(f"{'Valid' if validation else 'Test'} Acc:{correct/size:>7f}, Avg Loss: {test_loss/size:>7f}")

In [21]:
MUTE = False
EPOCH = 15
LR = 0.0001

torch.manual_seed(SEED)
np.random.seed(SEED)

optimizer = torch.optim.Adam(model.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model, loss_func, optimizer, mute = MUTE)
    test(validLoader, model, loss_func, validation = True)
    # test(testLoader, model, loss_func)


--------- Epoch #1 ---------
loss: 0.688457  [    0/ 4572]
loss: 0.682803  [  640/ 4572]
loss: 0.690039  [ 1280/ 4572]
loss: 0.670908  [ 1920/ 4572]
loss: 0.695890  [ 2560/ 4572]
loss: 0.672562  [ 3200/ 4572]
loss: 0.658255  [ 3840/ 4572]
loss: 0.719300  [ 4480/ 4572]
Valid Acc:0.555774, Avg Loss: 0.021309
--------- Epoch #2 ---------
loss: 0.674256  [    0/ 4572]
loss: 0.685002  [  640/ 4572]
loss: 0.656634  [ 1280/ 4572]
loss: 0.635314  [ 1920/ 4572]
loss: 0.678640  [ 2560/ 4572]
loss: 0.678888  [ 3200/ 4572]
loss: 0.732176  [ 3840/ 4572]
loss: 0.658299  [ 4480/ 4572]
Valid Acc:0.639108, Avg Loss: 0.020316
--------- Epoch #3 ---------
loss: 0.689377  [    0/ 4572]
loss: 0.629156  [  640/ 4572]
loss: 0.604820  [ 1280/ 4572]
loss: 0.693667  [ 1920/ 4572]
loss: 0.605697  [ 2560/ 4572]
loss: 0.558396  [ 3200/ 4572]
loss: 0.588147  [ 3840/ 4572]
loss: 0.710575  [ 4480/ 4572]
Valid Acc:0.631234, Avg Loss: 0.020083
--------- Epoch #4 ---------
loss: 0.658078  [    0/ 4572]
loss: 0.634144  [

## Test

In [22]:
test(testLoader, model, loss_func)

Test Acc:0.650919, Avg Loss: 0.019407
